![R-Kenntnis](../Pics/header.png "R-Kenntnis")

<img src="https://img.shields.io/badge/--Kenntnis-blue?style=flat-square&logo=r&logoColor=white"/><a href="https://ostfalia.de/w" target="_blank"><img src="https://img.shields.io/badge/Ostfalia-Fakultät%20W-blue?style=flat-square&logo=googlescholar&logoColor=white"/></a> <img src="https://img.shields.io/badge/Semester-WiSe2022%2F23-green?style=flat-square"/> <img src="https://img.shields.io/badge/Copyright-2020--22-orange?style=flat-square"/> <a href="https://creativecommons.org/licenses/by-nc-sa/4.0/" target="_blank"><img src="https://img.shields.io/badge/License-by--nc--sa-red?style=flat-square"/></a> 

<div class="alert alert-block alert-info">

* **Titel:** R-Kenntnis Lösung Übungen 09: Klassifikation und Clustering mit R
* **Autor:** Prof. Dr. Denis Royer
* **Datum:** 14.12.2022 (Version 2.0)

</div>

# BI - R-Kenntnis Lösung Übungen 09: Klassifikation und Clustering mit R

Dieses Übungsblatt bezieht sich auf den R-Kurs ***R-Kenntnis*** zur Vorlesung Business Intelligence im Wintersemester 2022/23.

Bitte lesen Sie die folgenden Kapitel und die enthaltenen Hinweise ***sorgfältig*** durch. Die Aufgaben sind zum Teil in den Kapiteln enthalten.

<div class="alert alert-block alert-warning">
<b>WICHTIG:</b> 
    
*Die Schritte im Source Code bauen zum Teil aufeinander auf. Sollten Sie Ihre RStudio oder JupyterLab Session schließen oder neu starten, so müssen Sie ggf. den **Code erneut ausführen**, bzw. die **Packages neu laden**.*
    
</div>

<div class="alert alert-block alert-info">
<b>Hinweis:</b> Weitere Hinweise und Quellen finden Sie <a href="../index.ipynb">auf der zenteralen Übersicht zu den Übungen</a>. </div>

## Vorbereitende Schritte für die Übung 09

<div class="alert alert-block alert-warning">
<b>Wichtig:</b> Bevor wir loslegen, müssen wir zunächst einmal ein paar vorbereitende Dinge erledigen:

* Die notwendigen Packages laden (bspw. `tidyverse`)
* Die übrigen Packages werden nach und nach in den Teilübungen geladen.

</div>

In [ ]:
library(tidyverse)
library(knitr)
library(ggplot2)
library(gridExtra)
library(caret)

# Klassifikation und Clustering mit R

In dieser Übung geht es um die Anwendung von Methoden zur Klassifikation und Clustering in ***R***. Als Datensatz für die Demonstration von Klassifikation und Clustering wird das Dataset `iris` benutz. 

Beim Iris Datensatz, handelt es sich um einen Datensatz mit 150 Beobachtungen von 4 Attributen von Schwertlilien. Gemessen wurden dabei jeweils die Breite und die Länge des Kelchblatts (Sepalum) sowie des Kronblatts (Petalum) in Zentimeter. Des Weiteren ist für jeden Datensatz die Art der Schwertlilie (Iris setosa, Iris virginica oder Iris versicolor) angegeben. Für jede Schwertlilienart liegen 50 Datensätze vor. 

![](../Pics/iris-machinelearning.png)

Dieser Datensatz wird - wie auch hier - in der Clusteranalyse bzw. in der Mustererkennung als Testdatensatz herangezogen, um aufgrund der gemessenen Attribute die Art der Schwertlilie automatisch zu erkennen. Der Datensatz eignet sich gerade deshalb so gut, weil eine Schwertlilienart aufgrund der Messungen sehr gut identifiziert werden kann, während die beiden anderen Arten überschneidende Wertebereiche in jeder Eigenschaft aufweisen. Außerdem ist eine Darstellung aller Informationen im dreidimensionalen Datenraum (z. B. mit einem 3D-Scatterplot) nicht mehr möglich, weshalb der Datensatz auch in der Informationsvisualisierung, die Daten ohne inhärenten Raumbezug analysiert, zur Anwendung kommt. 

In [ ]:
# Ausgabe der ersten Zeilen des iris Datensatzes
data("iris")
head(iris) 

## Grafische Exploration der Daten

Im Folgenden schauen wir uns die iris Daten einmal genauer an. Ein erster Ansatz in Richtung der Klassifikation und des Clustering ist es, sich die Daten und deren Verteilung pro Klasse einmal genauer anzuschauen. Im Beispiel geschieht dies für das Datum *Sepal.Width*.

In [ ]:
library(psych)

pairs.panels(iris[1:4],
             gap=0,
             bg = c("red","green","blue")[iris$Species],
             pch=21
)

In [ ]:
plot1 <- ggplot(data=iris, aes(Sepal.Length, fill = Species))+ 
  theme_bw()+
  geom_density(alpha=0.25)+
  labs(x = "Sepal.Length", title="Spezies vs. Sepal Length")


plot2 <- ggplot(data=iris, aes(x = Sepal.Width)) +
 stat_density(aes(ymax = ..density..,  ymin = -..density.., 
                       fill = Species, color = Species), 
                   geom = "ribbon", position = "identity") +
  facet_grid(. ~ Species) + coord_flip() + theme_bw()+labs(x = "Sepal Width", title="Spezies vs. Sepal Width")

grid.arrange(plot1, plot2, ncol=2)

# Mehr zum Thema Anordnen von plots und ggplot2 finden Sie hier: 
# https://stackoverflow.com/questions/1249548/side-by-side-plots-with-ggplot2

<div class="alert alert-block alert-info">
<b>Mehr zum Thema Visualisierung (iris und ggplot2) finden Sie hier:</b> 

* <https://www.kaggle.com/leolcling/visualizing-iris-datasets-with-r-ggplot2>
* <https://hub.packtpub.com/data-visualization-ggplot2/>
* <https://www.datanovia.com/en/lessons/introduction-to-ggplot2/>

</div>

## Klassifikation

Zunächst schauen wir uns den Bereich Klassifikation an - spezifisch die folgenden Aspekte:

* Erstellen von Trainings- und Testdatensätzen
* Recursive Partytioning (`party` - Package)
* Naive Baysian (`caret` - Package)
* k-nächste-Nachbarn (`class` - Package)

![Übungsaufgabe](../Pics/excercise.png "Übungsaufgabe")
***Aufgaben/Fragen:***

* Für die Klassifikation: Was sind die Werte für X, C, O, etc. (Klassifikationsfunktion $O(x_1, \dots , x_d) - f: X \rightarrow C$)?

<div class="alert alert-block alert-success">
<b>Lösungsansatz:</b></div>

**Grundsätzlich gilt:** Gegeben ist eine Menge $O$ von Objekten des Formats ($x_1, \dots, x_d$) mit Attributen $X_i$, $1≤ i≤ d$, und Klassenzugehörigkeit $c_i, c_i \in C = {c_1, \dots , c_k}$ 

Damit ergeben sich folgende Ausprägungen von X, C und O:

* **X:** Die Attribute *Sepal.Length, Sepal.Width, Petal.Length, Petal.Width*
* **C:** Spezies Zuordnung der Objekte: *setosa versicolor virginica*
* **O:** Die Daten im Datensatz iris

### Erstellen  von Trainings- und Testdatensätzen

Trainings- und Testdatensätze sind zwei verschiedene Teile eines Datensatzes, die beim maschinellen Lernen verwendet werden. Der Trainingsdatensatz wird verwendet, um ein maschinelles Lernmodell zu erstellen, indem es auf die Daten im Trainingsdatensatz trainiert wird. Der Testdatensatz wird verwendet, um das Modell zu bewerten, indem es auf die Daten im Testdatensatz angewendet wird.

Der Hauptunterschied zwischen Trainings- und Testdatensätzen besteht darin, dass der Trainingsdatensatz verwendet wird, um das Modell zu erstellen, während der Testdatensatz verwendet wird, um das Modell zu bewerten. Der Trainingsdatensatz sollte daher so ausgewählt werden, dass er dem Modell genügend Informationen liefert, um es zu trainieren, ohne dabei zu sehr von dem Testdatensatz abzugrenzen. Der Testdatensatz sollte hingegen so ausgewählt werden, dass er dem Modell genügend Herausforderungen stellt, um die Genauigkeit des Modells zu bewerten, ohne dabei zu sehr vom Trainingsdatensatz abzugrenzen.

Auf folgende Weise lassen sich Trainings- und Testdatensätzen aus dem `iris` Datensatz erzeugen:

In [ ]:
# Daten in 2 Subsets aufteilen: 
# - Trainingsdaten (70%) und 
# - Testdaten (30%); 

# Zufallsvariable so setzen, damit Ergebnisse reproduzierbar werden
set.seed(1234)

In [ ]:
# sample Funktion nutzen, um Trainings- und Testdatensatz zu erzeugen .
#
# Mit sample() können von einem Vektor Teilstichproben einer festgelegten 
# Länge mit und ohne Zurücklegen realsiert werden. Dies ist zum Beispiel 
# für Bootstrapping-Verfahren (Resampling) notwendig.

index <- sample(2, nrow(iris), replace = TRUE, prob = c(0.7, 0.3))

# Sample erzeugt einen neuen Vector "index" mit der Länge des
# Datensatzes "iris". Die Zahlen im Vector "index" entsprechen
# jeweils der Zuordnung zu den Trainings- und Testdaten.

In [ ]:
index
index[5]

In [ ]:
# Aufteilung der Daten mit der Hilfe der Variable "index"
# 70% Trainingsdaten --> Index == 1
# 30% Testdaten --> Index == 2
train.data <- iris[index == 1, ]
test.data <- iris[index == 2, ]

### Confusion Matrix

Eine Confusion Matrix ist ein Werkzeug zur Bewertung der Leistung eines Klassifikationsmodells. Sie zeigt die Anzahl der korrekt und falsch klassifizierten Datenpunkte in einem Klassifikationsproblem. Die Confusion Matrix enthält verschiedene Kennzahlen, die verwendet werden können, um die Leistung des Modells zu bewerten.

Die Confusion Matrix enthält im Allgemeinen vier Hauptkennzahlen: 

 * Die Anzahl der true positives (korrekt klassifizierte Datenpunkte, die tatsächlich in der positiven Klasse liegen)
 * Die Anzahl der false positives (falsch klassifizierte Datenpunkte, die tatsächlich in der negativen Klasse liegen)
 * Die Anzahl der true negatives (korrekt klassifizierte Datenpunkte, die tatsächlich in der negativen Klasse liegen) 
 * Die Anzahl der false negatives (falsch klassifizierte Datenpunkte, die tatsächlich in der positiven Klasse liegen).

Diese Kennzahlen können verwendet werden, um verschiedene Metriken zu berechnen, die die Leistung des Klassifikationsmodells bewerten, z.B. die Genauigkeit, die Sensitivität, die Spezifität und der F1-Score. Die Genauigkeit gibt an, wie viele der klassifizierten Datenpunkte insgesamt korrekt sind, während die Sensitivität und die Spezifität die Fähigkeit des Modells angeben, die positiven und negativen Klassen korrekt zu klassifizieren. Der F1-Score ist eine Kombination aus Sensitivität und Spezifität und gibt die allgemeine Leistung des Modells an.

![](../Pics/confusionmatrix.jpg)

In diesem Beispiel wird angenommen, dass das Klassifikationsmodell die Klasse "Positive" vorhersagt. Die Confusion Matrix zeigt die Anzahl der korrekt und falsch klassifizierten Datenpunkte in der Klasse "Positive" (RP und FP) und in der Klasse "Negative" (FN und RN). Aus dieser Confusion Matrix können verschiedene Metriken berechnet werden, um die Leistung des Modells zu bewerten. Zum Beispiel kann die Genauigkeit wie folgt berechnet werden:

Genauigkeit = (RP + RN) / (RP + RN + FP + FN)

Die Genauigkeit gibt an, wie viele der klassifizierten Datenpunkte insgesamt korrekt sind. In diesem Beispiel würde die Genauigkeit das Verhältnis der korrekt klassifizierten Datenpunkte zu allen klassifizierten Datenpunkten darstellen.

### Recursive Partytioning Klassifikation (`party` - Package)

Recursive Partytioning Klassifikation ist ein Verfahren zum Klassifizieren von Daten, bei dem ein Klassifikationsbaum verwendet wird, um die Datenpunkte in verschiedene Klassen einzuteilen. Der Klassifikationsbaum wird dabei durch rekursive Aufteilung der Daten in immer kleinere Teilmengen erstellt, bis jeder Datenpunkt in seiner eigenen Teilmenge enthalten ist.

Das Verfahren beginnt damit, dass alle Datenpunkte in einer einzigen Teilmenge zusammengefasst werden. Anschließend werden die Datenpunkte auf der Grundlage eines bestimmten Kriteriums in zwei Teilmengen aufgeteilt. Dieser Schritt wird wiederholt, bis jeder Datenpunkt in seiner eigenen Teilmenge enthalten ist. Die Entscheidung, wie die Datenpunkte in Teilmengen aufgeteilt werden, wird anhand eines bestimmten Kriteriums getroffen, das normalerweise auf dem Verhältnis von Klassen in den Teilmengen basiert.

Ein Vorteil von Recursive Partytioning Klassifikation ist, dass es ein einfaches und effektives Verfahren ist, das gut für große Datensätze geeignet ist. Ein Nachteil ist jedoch, dass es leicht zu Overfitting neigt, insbesondere bei komplexen Klassifikationsproblemen.

In [ ]:
##########################################################
#### Klassifikation:
#### Recursive Partytioning (party)
#### Entscheidungsbaum erstellen
##########################################################

library(party)

In [ ]:
# Erstellung eines Conditional Inference Trees
# Recursive Partytioning
myFormula <- Species ~ Sepal.Length + Sepal.Width + Petal.Length + Petal.Width
iris.ctree <- ctree(myFormula, data = train.data)

In [ ]:
# check the prediction
table(predict(iris.ctree), train.data$Species)
print(iris.ctree)
plot(iris.ctree)
plot(iris.ctree, type = "simple")

In [ ]:
#### Modell überprüfen

# Vorhersage testen
testPred <- predict(iris.ctree, newdata = test.data)
tab.dtree <- table(testPred, test.data$Species)
# Anzahl richtiger Entscheidungen
class.right <- sum(diag(tab.dtree))
# Anzahl falscher Entscheidungen
class.table<- sum(tab.dtree)

In [ ]:
tab.dtree
precision.dtree <- class.right/(class.table)
precision.dtree * 100

In [ ]:
# Wie gut ist die Vorhersagekraft?
confusionMatrix(data=testPred, reference = test.data$Species)

### Naive Baysian Klassifikation (`caret` - Package)

Ein Bayes-Klassifikator, ist ein aus dem Satz von Bayes hergeleiteter Klassifikator. Er ordnet jedes Objekt der Klasse zu, zu der es mit der größten Wahrscheinlichkeit gehört, oder bei der durch die Einordnung die wenigsten Kosten entstehen. Formal handelt es sich um eine mathematische Funktion, die jedem Punkt eines Merkmalsraums eine Klasse zuordnet.

Um den Bayes-Klassifikator zu definieren, wird ein Kostenmaß benötigt, das jeder möglichen Klassifizierung Kosten zuweist. Der Bayes-Klassifikator ist genau derjenige Klassifikator, der die durch alle Klassifizierungen entstehenden Kosten minimiert. Das Kostenmaß wird gelegentlich auch Risikofunktion genannt; man sagt dann, der Bayes-Klassifikator minimiere das Risiko einer Fehlentscheidung und sei über das *minimum-risk*-Kriterium definiert.

In [ ]:
##########################################################
#### Klassifikation:
#### Naive Baysian (caret)
##########################################################

library(caret)
library(psych)
library(klaR)

In [ ]:
iris.caret <- iris

iris.caret.features = iris.caret[,-5]
irisiris.caret.classes = iris.caret$Species

iris.caret.fit = train(iris.caret.features,
                       irisiris.caret.classes,
                       'nb',
                       trControl=trainControl(method='cv',number=10))

In [ ]:
iris.caret.fit

predict(iris.caret.fit$finalModel,iris.caret.features)

In [ ]:
tab.caret <- table(predict(iris.caret.fit$finalModel,iris.caret.features)$class,irisiris.caret.classes)
# Anzahl richtiger Entscheidungen
class.right <- sum(diag(tab.caret))
# Anzahl falscher Entscheidungen
class.table<- sum(tab.caret)

In [ ]:
# Wie gut ist die Vorhersagekraft?
precision.caret <-class.right/(class.table) # --> 96%
precision.caret * 100

naive_iris <- NaiveBayes(iris$Species ~ ., data = iris)
plot(naive_iris )

### k-nächste-Nachbarn Klassifikation (`class` - Package)

Die Nächste-Nachbarn-Klassifikation ist eine parameterfreie Methode zur Schätzung von Wahrscheinlichkeitsdichtefunktionen. Der daraus resultierende k-Nearest-Neighbor-Algorithmus (KNN, zu Deutsch "k-nächste-Nachbarn-Algorithmus") ist ein Klassifikationsverfahren, bei dem eine Klassenzuordnung unter Berücksichtigung seiner k nächsten Nachbarn vorgenommen wird. Der Teil des Lernens besteht aus simplem Abspeichern der Trainingsbeispiele, was auch als lazy learning ("träges Lernen") bezeichnet wird.

In [ ]:
##########################################################
#### Klassifikation:
#### k-Nearest-Neighbor (KNN)
##########################################################

library(class)
iris.knn.fit <- knn(train= train.data[,-5],test=test.data[,-5], cl= train.data[,5],k=13)
table(factor(iris.knn.fit))

iris.knn.fit

In [ ]:
#### Modell überprüfen

# Vorhersage testen
testPred <- predict(iris.ctree, newdata = test.data)
tab.knn <- table(test.data[,5],iris.knn.fit)
# Anzahl richtiger Entscheidungen
class.right <- sum(diag(tab.knn))
# Anzahl falscher Entscheidungen
class.table<- sum(tab.knn)

In [ ]:
# Wie gut ist die Vorhersagekraft?
precision.knn <- class.right/(class.table)
precision.knn * 100

## Clustering

Im nächsten Schritt geht es darum, die Daten aus `iris` automatisiert in Cluster zu teilen:

In [ ]:
# Ausgabe Originaldaten
plot(iris[,1:4], col = iris$Species)

### Clustering mit k-means

Clustering mit k-means ist ein Verfahren zum Gruppieren von Daten in Cluster. Das Ziel des k-means-Clustering ist es, die Daten so zu gruppieren, dass die Datenpunkte innerhalb eines Clusters möglichst ähnlich sind, während die Datenpunkte in verschiedenen Clustern möglichst unterschiedlich sind.

Das k-means-Verfahren beginnt damit, dass zunächst zufällig k-Cluster ausgewählt werden. Jeder Cluster wird durch seinen Mittelwert, auch als Centroid bezeichnet, repräsentiert. Anschließend werden die Datenpunkte den Clustern zugeordnet, indem jedem Datenpunkt der Cluster zugewiesen wird, dessen Centroid ihm am nächsten ist. Sobald die Zuordnungen feststehen, werden die Centroids neu berechnet, indem der Mittelwert aller Datenpunkte in jedem Cluster berechnet wird. Dieser Schritt wird wiederholt, bis die Centroids sich nicht mehr verändern oder eine vorher festgelegte Anzahl von Iterationen erreicht wurde.

Insgesamt ist k-means-Clustering ein einfaches und effektives Verfahren zum Gruppieren von Daten, das jedoch auch seine Grenzen hat. Ein wichtiger Nachteil des k-means-Verfahrens ist, dass es davon ausgeht, dass die Cluster kugelförmig sind und dass alle Cluster die gleiche Varianz haben. In der Praxis können die Daten jedoch in Clustern mit unterschiedlicher Form und Varianz vorkommen, was dazu führen kann, dass das k-means-Verfahren nicht immer die besten Ergebnisse liefert.

In [ ]:
##########################################################
#### Clustering:
#### Bsp.: k-means
##########################################################

# Daten laden
iris.clustering <- iris

# nicht so wichtig für das iris Dataset (metrisch Skaliert)
# Aber bei metrischen Skalen wichtig (Alter, Abstand)
# Weiterhin: Nur kontinuierliche Daten sollen sakliert werden
# iris.clustering.scaled <- scale(iris.clustering[,-5])

# Es gibt verschiedene Algorithmen
# k-means wird hier genutzt (Hierarchisches Clustering)


fit.K <- kmeans(iris.clustering[,-5], 3)
fit.K

plot(iris, col = fit.K$cluster)

In [ ]:
# bessere Annährung von k suchen
# Hierfür prüfn wir einfach die Parameter für k
# von 1 bis 10 einmal durch - dies geschieht mit einer 
# for-Schleife

k <- list()
for (i in 1:10){
  k[[i]] <- kmeans(iris.clustering[,-5], i)
  
}

In [ ]:
# Als nächstes prüfen wir, wie gut die Abdeckung der Cluster ist
# und geben das Ergebnis als Plot einmal aus

k.betweenss_toss <- list()

for(i in 1:10){
  k.betweenss_toss[[i]] <- k[[i]]$betweenss/k[[i]]$totss
}

In [ ]:
plot(1:10, 
     k.betweenss_toss, 
     type = "b",
     ylab="Between SS / Total SS",
     xlab="Clusters (k)")

In [ ]:
par(mfrow = c(2, 2))
# Analyse der Cluster für k = 1-4
for(i in 1:4){
  plot(iris, 
       col = k[[i]]$cluster,
       main = paste ("Plot für k=", i))
  }

In [ ]:
k <- list()
for (i in 1:10){
  k[[i]] <- kmeans(iris[,-5], i)
  
}

In [ ]:
# Als nächstes prüfen wir, wie gut die Abdeckung der Cluster ist
# und geben das Ergebnis als Plot einmal aus

k.betweenss_toss <- list()

for(i in 1:10){
  k.betweenss_toss[[i]] <- k[[i]]$betweenss/k[[i]]$totss
}

plot(1:10, 
     k.betweenss_toss, 
     type = "b",
     ylab="Between SS / Total SS",
     xlab="Clusters (k)",
     main = "Festlegung von k")

### Hierarchisches Clustering

Hierarchisches Clustering ist ein Verfahren zum Gruppieren von Daten, bei dem die Datenpunkte zunächst alle in eigene Cluster eingeordnet werden und anschließend sukzessive zu immer größeren Clustern zusammengefasst werden. Dieses Verfahren kann dazu verwendet werden, um die Struktur der Daten zu entdecken und die Beziehungen zwischen den Datenpunkten aufzudecken.

Es gibt zwei Haupttypen hierarchischen Clustering: agglomeratives Clustering und divisives Clustering. Beim agglomerativen Clustering werden die Datenpunkte zunächst in eigene Cluster eingeordnet und anschließend sukzessive zusammengefasst, bis alle Datenpunkte in einem einzigen Cluster sind. Beim divisiven Clustering werden die Datenpunkte zunächst in einem einzigen Cluster zusammengefasst und anschließend sukzessive in immer kleinere Cluster aufgeteilt.

Ein wichtiger Vorteil von hierarchischem Clustering ist, dass es keine vorherige Angabe der Anzahl der Cluster erfordert, wie dies beim k-means-Clustering der Fall ist. Stattdessen wird die Anzahl der Cluster aus den Daten selbst abgeleitet, was es zu einem flexiblen Verfahren macht. Ein Nachteil von hierarchischem Clustering ist jedoch, dass es in der Regel langsamer ist als andere Clustering-Verfahren, insbesondere bei großen Datensätzen.

In [ ]:
##########################################################
#### Clustering:
#### Bsp.: Hierarchisches Clustering
##########################################################

d <- dist(iris.clustering[,-5])
# Parameter --> Typ von Clustering
fit.H <- hclust(d, "ward.D2")

In [ ]:
# Dendrogramm ausgeben
plot(fit.H,
     main = "Dendrogramm")
rect.hclust(fit.H, k=3, border = "red")

In [ ]:
# Um die Zugehörigkeit eines Datensatzes zu einem Cluster 
# zu finden, müssen diese einmal mit cutree() in eine Variable
# Ausgelesen werden

clusters.H <- cutree(fit.H, 3)
clusters.H

plot(iris, col = clusters.H)

### Weiter Clustering Methoden

In [ ]:
##########################################################
#### Clustering:
#### Bsp.: Model-based Clustering
##########################################################

library(mclust)
fit.M <- Mclust(iris.clustering[,-5])
fit.M

plot(fit.M)

In [ ]:
##########################################################
#### Clustering:
#### Bsp.: Density-based Clustering
##########################################################

library(dbscan)

kNNdistplot(iris.clustering[,-5], k=3)
abline(h = 0.7, col = "red", lty = 2)
# minPts --> Anzahl Variablen + 1 ==> 5
fit.D <- dbscan(iris.clustering[,-5], eps = 0.7, minPts = 5)
fit.D
plot(iris, col = fit.D$cluster)

# Lösungen

Die Lösungen sind in diesem Arbeitsblatt schon direkt enthalten.